# PROCEDURE Demo

- Prozeduren erleichtern Datenpflege-Routinen. Der Prozeduraufruf erspart das Erstellen evtl. komplexer INSERT-, UPDATE- oder DELETE-Anweisungen.
- Außerdem erhöht die Verwendung von Prozeduren die Sicherheit, wenn Benutzer und Anwendungen Berechtigungen zum Aufrufen von Prozeduren (CONTROL), aber nicht Berechtigungen zum Ändern von Tabelleninhalten (INSERT, UPDATE, DELETE) erhalten.
- Prozeduren werden mit dem Befehl EXEC (kurz für execute) aufgerufen. Parameter werden durch Komma getrennt übergeben.
- Wenn beim Aufruf ein Parameter fehlt oder einen unpassenden Datentyp hat, wird die Prozedur nicht ausgeführt.

**Beispiel**: Um das Budget eines Projekts aufzustocken oder zu reduzieren, wird die Prozedur dbo.usp\_BudgetAnpassen aufgerufen.

**Voraussetzung:**  Die Tabelle Projekt benötigt die Spalten, die im Aufbau der Trigger-Demo hinzugefügt wurden.

In [ ]:
USE NormalisierungsDemo;

In [ ]:
CREATE OR ALTER PROCEDURE dbo.usp_BudgetAnpassen (
    @Projektbezeichnung VARCHAR(120),
    @BudgetÄnderung DECIMAL(9,2)
)
AS 
BEGIN
    SET NOCOUNT ON;
    BEGIN TRY
        DECLARE @ProjNr INT, @Budget DECIMAL(9,2), @Personalkostenstand DECIMAL(9,2);
        -- Das Projekt muss anhand der Bezeichnung (Beschreibung) identifiziert werden.
        SELECT @ProjNr = ProjNr, @Budget = Personalbudget, @Personalkostenstand = Personalkostenstand
            FROM Projekt
            WHERE Beschreibung = @Projektbezeichnung;
        -- Projekt gefunden?    
        IF @ProjNr IS NULL THROW 50000, 'Fehler: Projekt nicht gefunden', 1;
        -- Zielbudget noch nicht überschritten?
        IF (@BudgetÄnderung < 0 AND 
            @Budget + @BudgetÄnderung < @Personalkostenstand ) THROW 50000, 'Fehler: Kosten bereits über Budget!', 1;
        -- Jetzt kann das Budget angepasst werden
        UPDATE Projekt
            SET Personalbudget += @BudgetÄnderung
            OUTPUT @Projektbezeichnung AS Projekt 
                , deleted.Personalbudget AS [Altes Budget]
                , inserted.Personalbudget AS [Neues Budget]
                , deleted.Personalkostenstand AS [Bisherige Kosten]
            WHERE ProjNr = @ProjNr;  
         PRINT 'Das Projektbudget wurde erfolgreich angepasst.';      
    END TRY
    BEGIN CATCH
        PRINT 'Die Budgetanpassung war NICHT möglich!';
        THROW;
    END CATCH
    SET NOCOUNT OFF;
END;

## Es folgen Testfälle für den Funktionsaufruf

1. Ungültiger Aufruf mit fehlendem Parameter @BudgetÄnderung.
2. Ungültiger Aufruf mit falschem Datentyp im Parameter @BudgetÄnderung.
3. Fehler durch vertippte Projektbezeichnung.
4. Fehler, weil Kosten bereits über neuem Zielbudget liegen.
5. Erfolgreicher Prozeduraufruf zur Erhöhung des Budgets.

In [ ]:
-- Die Prozedur oder Funktion "usp_BudgetAnpassen" erwartet den @BudgetÄnderung-Parameter, der nicht bereitgestellt wurde.
EXEC dbo.usp_BudgetAnpassen 'Verkaufsmesse';

In [ ]:
EXEC dbo.usp_BudgetAnpassen 'Verkaufsmesse', 'das Doppelte'; -- Fehler beim Konvertieren des varchar-Datentyps in decimal.

In [ ]:
EXEC dbo.usp_BudgetAnpassen 'Verkaufsmessssse', 1000; -- Fehler: Projekt nicht gefunden

In [ ]:
EXEC dbo.usp_BudgetAnpassen 'Verkaufsmesse', -5000; -- Fehler: Kosten bereits über Budget!

In [ ]:
EXEC dbo.usp_BudgetAnpassen 'Verkaufsmesse', 1000; -- Budgeterhöhung ok